In [7]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import StratifiedKFold
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelBinarizer
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import sys

import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('../data/Processed/train.csv')
X_train = train.loc[:,[c for c in train.columns if c != 'patient_id']]
y_train = pd.read_csv('../data/raw/train_labels.csv', usecols=['heart_disease_present'])
test = pd.read_csv('../data/Processed/test.csv')
X_test = test.loc[:,[c for c in test.columns if c != 'patient_id']]

In [22]:
model_name = 'lr_1'

params = {'tol':0.01,
          'C':10,
          'max_iter':10000,
          'n_jobs':-1,
          'verbose':5,
          'random_state': 42,
         'penalty': 'l2'}


lr_model = LogisticRegression(**params)

In [23]:
train_ids = X_train.index
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [26]:
LabelBinarizer(y_val)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [24]:
counter = 1
y_preds = np.zeros(X_test.shape[0])
be = 0
for train_index, test_index in skf.split(train_ids, y_train):
    print('Fold {}\n'.format(counter))

    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train.iloc[train_index], y_train.iloc[test_index]

    lr_model.fit(X_fit,
                  y_fit)
    
    
    preds = lr_model.predict_proba(X_test)
    y_preds += preds[:,1]
    
    fold_score = log_loss(preds, LabelBinarizer(y_val['heart_disease_present']))
    be += fold_score
    print(fold_score)

    del X_fit
    del X_val
    del y_fit
    del y_val
    del train_index
    del test_index
    gc.collect()

#     print('Guardamos el modelo')
#     joblib.dump(lgb_model, '../saved_models/{}_{}.pkl'.format(model_name, counter))

#     ft_importances += lgb_model.feature_importances_

    counter += 1

y_preds = y_preds / (counter-1)

print('\n\nBEST SCORE MEAN:', be / (counter-1))


Fold 1

[LibLinear]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [21]:
y_preds[:5]

array([0.36670871, 0.10192963, 0.94859829, 0.06797782, 0.96176598])

In [54]:
sub = pd.DataFrame({'patient_id': test['patient_id'], 'heart_disease_present': y_preds})

In [55]:
sub.to_csv('../submissions/lgbm_1.csv', index=False)